# GPU-Jupyter

This Jupyterlab Instance is connected to the GPU via CUDA drivers. In this notebook, we test the installation and perform some basic operations on the GPU.

## Test GPU connection

### Using the following command, your GPU type and its NVIDIA-SMI driver version should be listed:

In [1]:
!nvidia-smi

Mon Apr  8 16:12:04 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.05              Driver Version: 545.84       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060 ...    On  | 00000000:01:00.0 Off |                  N/A |
| N/A   39C    P8              10W /  60W |    105MiB /  6144MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### Now, test if PyTorch can access the GPU via CUDA:

In [2]:
from __future__ import print_function
import numpy as np
import torch

print(torch.cuda.is_available())

a = torch.rand(5, 3)
a

True


tensor([[0.3820, 0.9186, 0.0873],
        [0.0677, 0.3465, 0.2376],
        [0.3284, 0.1974, 0.0858],
        [0.6180, 0.1698, 0.6550],
        [0.8948, 0.0876, 0.5775]])

### Now, test if Tensorflow can access the GPU via CUDA:

In [3]:
import tensorflow as tf
# NUMA warnings are no problem
print(tf.test.is_built_with_cuda(), tf.config.list_physical_devices('GPU'))

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

2024-04-08 16:12:05.608277: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-08 16:12:06.372910: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


True [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5065807306722951698
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3774873600
locality {
  bus_id: 1
  links {
  }
}
incarnation: 18240998582538193189
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


2024-04-08 16:12:07.257260: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-08 16:12:07.258537: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-08 16:12:07.258578: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-08 16:12:07.261418: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-08 16:12:07.261470: I external/local_xla/xla/stream_executor

In [4]:
# Test GPU-calculation in TF
with tf.device('/gpu:0'):
    a = tf.Variable(tf.random.uniform(shape=(100, 100)), name="a")
    b = tf.matmul(a, a)
    del a

np.mean(b)

2024-04-08 16:12:07.398413: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-08 16:12:07.398507: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-08 16:12:07.398537: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-08 16:12:07.398712: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-08 16:12:07.398727: I tensorflow/core/common_runtime/gpu/gpu

24.57181

## Performance test

#### Now we want to know how much faster a typical operation is using GPU. Therefore we do the same operation in numpy, PyTorch and PyTorch with CUDA. The test operation is the calculation of the prediction matrix that is done in a linear regression.

### 1) Numpy

In [5]:
x = np.random.rand(10000, 256)

In [6]:
%%timeit
H = x.dot(np.linalg.inv(x.transpose().dot(x))).dot(x.transpose())

666 ms ± 75.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 2) PyTorch

In [7]:
x = torch.rand(10000, 256)

In [8]:
%%timeit
# Calculate the projection matrix of x on the CPU
H = x.mm( (x.t().mm(x)).inverse() ).mm(x.t())

275 ms ± 12.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 3) PyTorch on GPU via CUDA

In [9]:
# let us run this cell only if CUDA is available
# We will use ``torch.device`` objects to move tensors in and out of GPU
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    x = torch.rand(10000, 256, device=device) # directly create a tensor on GPU
    y = x.to(device)                       # or just use strings ``.to("cuda")``
    print(x[0:5, 0:5])
    print(y.to("cpu", torch.double)[0:5, 0:5])

tensor([[0.3950, 0.1953, 0.3265, 0.5156, 0.5355],
        [0.2038, 0.3776, 0.9295, 0.4246, 0.4888],
        [0.4108, 0.8438, 0.9092, 0.9224, 0.2784],
        [0.6929, 0.9316, 0.6724, 0.0748, 0.4366],
        [0.4430, 0.9733, 0.1734, 0.9995, 0.0928]], device='cuda:0')
tensor([[0.3950, 0.1953, 0.3265, 0.5156, 0.5355],
        [0.2038, 0.3776, 0.9295, 0.4246, 0.4888],
        [0.4108, 0.8438, 0.9092, 0.9224, 0.2784],
        [0.6929, 0.9316, 0.6724, 0.0748, 0.4366],
        [0.4430, 0.9733, 0.1734, 0.9995, 0.0928]], dtype=torch.float64)


In [10]:
%%timeit
# Calculate the projection matrix of x on the GPU
H = x.mm( (x.t().mm(x)).inverse() ).mm(x.t())

55.3 ms ± 161 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Exhaustive Testing on GPU

In [11]:
# let us run this cell only if CUDA is available
# We will use ``torch.device`` objects to move tensors in and out of GPU
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    x = torch.rand(10000, 10, device=device) # directly create a tensor on GPU

In [12]:
if torch.cuda.is_available():
    y = x.to(device)                       # or just use strings ``.to("cuda")``
    print(x[0:5, 0:5])

tensor([[0.4825, 0.5817, 0.0127, 0.7462, 0.1225],
        [0.5351, 0.1582, 0.7482, 0.5826, 0.0507],
        [0.1559, 0.9457, 0.6242, 0.7610, 0.2080],
        [0.6010, 0.4131, 0.8429, 0.5539, 0.0943],
        [0.3711, 0.0609, 0.5242, 0.5414, 0.9859]], device='cuda:0')


In [13]:
if torch.cuda.is_available():
    # Here is the memory of the GPU a border.
    # A matrix with 100000 lines requires 37 GB, but only 8 GB are available.
    H = x.mm( (x.t().mm(x)).inverse() ).mm(x.t())

In [14]:
if torch.cuda.is_available():
    print(H[0:5, 0:5])

tensor([[ 1.2062e-03, -3.5245e-04,  6.0149e-04,  5.5662e-04, -5.2612e-04],
        [-3.5245e-04,  1.0502e-03, -1.2980e-04,  7.1976e-05,  2.5712e-04],
        [ 6.0149e-04, -1.2980e-04,  1.2645e-03,  4.9298e-04, -2.6166e-04],
        [ 5.5662e-04,  7.1976e-05,  4.9298e-04,  9.1148e-04, -2.7401e-04],
        [-5.2612e-04,  2.5712e-04, -2.6166e-04, -2.7401e-04,  1.0023e-03]],
       device='cuda:0')


In [15]:
if torch.cuda.is_available():
    # This operation is difficult, as an symmetric matrix is transferred
    # back to the CPU. Is possible up to 30000 rows.
    print(H.to("cpu", torch.double)[0:5, 0:5])

tensor([[ 1.2062e-03, -3.5245e-04,  6.0149e-04,  5.5662e-04, -5.2612e-04],
        [-3.5245e-04,  1.0502e-03, -1.2980e-04,  7.1976e-05,  2.5712e-04],
        [ 6.0149e-04, -1.2980e-04,  1.2645e-03,  4.9298e-04, -2.6166e-04],
        [ 5.5662e-04,  7.1976e-05,  4.9298e-04,  9.1148e-04, -2.7401e-04],
        [-5.2612e-04,  2.5712e-04, -2.6166e-04, -2.7401e-04,  1.0023e-03]],
       dtype=torch.float64)
